# What is Iterative Retrieval in Agentic RAG?
Combined both Iterative And Self reflection

Definition:
Iterative Retrieval is a dynamic strategy where an AI agent doesn't settle for the first batch of retrieved documents. Instead, it evaluates the adequacy of the initial context, and if necessary, it:

- Refines the query,
- Retrieves again,
- Repeats the process until it’s confident enough to answer the original question.

# Why Use It?
In standard RAG:

- A single retrieval step is done, and the LLM uses it to answer.
- If the documents were incomplete or irrelevant, the answer may fail.

In Iterative RAG:

- The agent reflects on the retrieved content and the answer it produced.
- If it’s unsure, it can refine its search (like a human researcher would).

In [24]:
import os
from typing import List
from pydantic import BaseModel
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader,WebBaseLoader
from langgraph.graph import StateGraph, END
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders.youtube import YoutubeLoader
from langchain_community.document_loaders import ArxivLoader
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langgraph.graph import StateGraph, END

In [ ]:
import google.generativeai as genai
import os

# --- IMPORTANT ---
# Make sure you configure your API key *before* running
genai.configure(api_key="your_api_key") 

class GeminiLLM:
    def __init__(self, model: str):
        # 1. Initialize the GenerativeModel object
        try:
            self.model = genai.GenerativeModel(model)
        except Exception as e:
            print(f"Error initializing model: {e}")
            print("Please ensure your API key is set and the model name is correct.")
            raise

    def invoke(self, prompt: str):
        # 2. Use the .generate_content() method
        response = self.model.generate_content(prompt)
        
        # 3. Access the text using response.text
        
        # Mimic LangChain's .content
        class Result:
            def __init__(self, content):
                self.content = content
        
        # 4. Pass the text content to your Result class
        return Result(content=response.text)

# --- Usage ---

# Note: "gemini-2.5" might not be a valid model name.
# Using "gemini-1.5-flash" as a common, valid example.
try:
    # Try this model name if 'gemini-1.5-flash' still fails after upgrading
    llm = GeminiLLM(model="gemini-2.5-flash")
    prompt = "hello"
    answer = llm.invoke(prompt).content
    print(answer)

except Exception as e:
    print(f"\nAn error occurred during invocation: {e}")

Hello! How can I help you today?


# load the text then split then embed then store

In [26]:
docs = TextLoader("data_science_interview_cleaned.txt",encoding="utf-8").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
doc_splits = text_splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(doc_splits, embeddings)
retriever = vectorstore.as_retriever()



### Define Agent State

| Field              | Type             | Default      | Meaning                                                                                                                                  |
| ------------------ | ---------------- | ------------ | ---------------------------------------------------------------------------------------------------------------------------------------- |
| `question`         | `str`            | *(required)* | The **original user query**. This is what the user initially asked.                                                                      |
| `refined_question` | `str`            | `""`         | The **improved or reformulated question**, after the system refines it (e.g., clarifying or expanding the query).                        |
| `retrieved_docs`   | `List[Document]` | `[]`         | A **list of documents** retrieved from external sources (e.g., vector store, database, web). Each `Document` contains text and metadata. |
| `answer`           | `str`            | `""`         | The **generated answer** after combining or synthesizing information from retrieved documents.                                           |
| `verified`         | `bool`           | `False`      | Indicates whether the answer has been **verified** (for accuracy or completeness) in the iterative loop.                                 |
| `attempts`         | `int`            | `0`          | Counts how many **iterations or attempts** have been made to refine and verify the answer.                                               |


In [ ]:
class IterativeRAGState(BaseModel): # pydantic model 
    question: str
    refined_question:str=""
    retrieved_docs:List[Document]=[]
    answer:str=""
    verified:bool=False
    attempts:int=0
    

### Retrieve Node

In [ ]:
### Retrieve Node
def retrieve_docs(state: IterativeRAGState) -> IterativeRAGState:
    query = state.refined_question or state.question # Use refined_question if it exists (non-empty), otherwise use the original question.
    docs = retriever.invoke(query)
    return state.model_copy(update={"retrieved_docs": docs})


### Reflect And Verify

In [ ]:
### Reflect And Verify
def generate_answer(state: IterativeRAGState) -> IterativeRAGState:
    
    context = "\n\n".join(doc.page_content for doc in state.retrieved_docs) # Extracts their text (page_content) and joins them with two line breaks (\n\n) for readability.
    prompt = f"""Use the following context to answer the question:

Context:
{context} # The result is one big context block — all the relevant text that the LLM will use to answer the question.

Question:
{state.question}
"""
    response = llm.invoke(prompt.strip()).content.strip() # .content extracts the text reply from that object. .strip() cleans up any leading/trailing whitespace.
    return state.model_copy(update={"answer": response, "attempts": state.attempts + 1}) #Updates two fields:answer → stores the new LLM-generated answer.attempts → increments by 1 (tracks how many times the system tried to generate or refine an answer).

## Reflect on answer

In [ ]:
## Reflect on answer
def reflect_on_answer(state: IterativeRAGState) -> IterativeRAGState:
    
    prompt = f"""
Evaluate whether the answer below is factually sufficient and complete.

Question: {state.question}
Answer: {state.answer}

Respond 'YES' if it's complete, otherwise 'NO' with feedback.
"""
    feedback = llm.invoke(prompt).content.lower()
    verified = "yes" in feedback # Respond "YES" if the answer is complete, otherwise "NO" with feedback.
    return state.model_copy(update={"verified": verified})


## Refine query


In [31]:
## Refine query
def refine_query(state: IterativeRAGState) -> IterativeRAGState:
    
    prompt = f"""
The answer appears incomplete. Suggest a better version of the query that would help retrieve more relevant context.

Original Question: {state.question}
Current Answer: {state.answer}
"""
    new_query = llm.invoke(prompt).content.strip()
    return state.model_copy(update={"refined_question": new_query})


In [ ]:
builder = StateGraph(IterativeRAGState)

builder.add_node("retrieve", retrieve_docs)
builder.add_node("answer", generate_answer)
builder.add_node("reflect", reflect_on_answer)
builder.add_node("refine", refine_query)

builder.set_entry_point("retrieve")
builder.add_edge("retrieve", "answer")
builder.add_edge("answer", "reflect")



"""s.verified (answer is verified as good)	→ Go to END (stop)
  🚫 s.attempts >= 2 (too many tries)	→ Also go to END (stop loop)
  ❌ Otherwise	→ Go to "refine" (refine query and try again)"""

builder.add_conditional_edges(   
    "reflect", 
    lambda s: END if s.verified or s.attempts >= 2 else "refine"
)

builder.add_edge("refine", "retrieve")
builder.add_edge("answer", END)

graph = builder.compile()


In [33]:
query = "data alysis science better" # wrong spellings to test the agenticrag pipeline

initial_state= IterativeRAGState(question=query)
final=graph.invoke(initial_state)

print("final answer: \n",final["answer"])
print("verified:\n",final["verified"])
print("attempts:\n",final["attempts"])

final answer: 
 Based on the provided context, the question "data alysis science better" seems to be asking about the relationship or comparison between data analysis and data science.

The text does not present data analysis as being "better" than data science, but rather depicts **data analysis as a critical component and set of activities within the broader field of data science.**

Here's how the context positions them:

*   **Data Science:** Described as a growing, multidisciplinary field focused on using cutting-edge technologies to tackle real-world challenges and derive value from data (the "new oil"). A skilled Data Scientist combines technical skills (parsing data, creating models) with business sense.
*   **Data Analysis:** Mentioned as a specific task or process. It's part of "data processing" ("Exploring, mining, and analyzing data are all tasks that data processing does"). It involves using "statistical and mathematical analysis" to "provide a summary of the data's insigh

In [34]:
query = "data analysis data science which is better"

initial_state= IterativeRAGState(question=query)
final=graph.invoke(initial_state)

print("final answer: \n",final["answer"])
print("verified:\n",final["verified"])
print("attempts:\n",final["attempts"])

final answer: 
 The provided context does not state that one is "better" than the other. Instead, it highlights their differences in focus and methodology:

*   **Data Science** is concerned with **predictive modeling**, altering data to derive useful insights, fostering innovation by answering future-oriented questions, and solving complex issues using a wide array of mathematical and scientific tools. It's a broad field requiring mastery of disciplines like math, statistics, programming, and machine learning.
*   **Data Analytics** is concerned with **verifying current hypotheses and facts**, answering questions for efficient business decision-making, and removing current meaning from past context.

Essentially, data science looks more towards the future and innovation through prediction, while data analytics focuses on understanding the present and past to inform current decisions. They serve different, yet complementary, purposes within an organization.
verified:
 True
attempts:
 2